Sự thành công của Word2vec đã được chứng minh trong rất nhiều công trình NLP. 

Nhắc lại về Word2vec, nó sử dụng 1 tập copus, qua một mạng Neural biểu diễn các word thành các vector, các vector giữ lại được tính chất ngữ nghĩa. Tức các từ mang ý nghĩa similar với nhau thì gần nhau trong không gian vector. Trong NLP, đây một trong những phương thức của **word embedding**. Word2vec hiện nay được sử dụng hết sức rộng rãi.

Với Doc2vec, ngoài từ (word), ta còn có thể biểu diễn các câu (sentences) thậm chí 1 đoạn văn bản (document). Khi đó, bạn có thể dễ dàng vector hóa cả một đoạn văn bản thành một vector có số chiều cố định và nhỏ, từ đó có thể chạy bất cứ thuật toán classification cơ bản nào trên các vector đó. 

Trong notebook này, mình sẽ giới thiệu rất cơ bản basic concept để các bạn có thể hình dung Sentiment sử dụng Doc2vec như thế nào.


# Doc2vec

Doc2vec được giới thiệu bởi Google ([https://arxiv.org/pdf/1507.07998v1.pdf](https://arxiv.org/pdf/1507.07998v1.pdf)), cũng như Word2vec, có 2 model chính là: **DBOW** và **DM**

*  **DBOW** (distributed bag of words): Mô hình này đơn giản là không quan tâm thứ tự các từ, training nhanh hơn, không sử dụng local-context/neighboring. Mô hình chèn thêm 1 "word" là ParagraphID, ParagraphID này đại diện cho văn bản được training. Sau khi training xong có thể hiểu các vector ParagraphID này là vector embedded của các văn bản. Hình ảnh được mô tả trong bài báo:

![DBOW](doc2vec_dbow.jpg)

* **DM** (distributed memory): xem một paragraph là một từ, sau đó nối từ này vào tập các từ trong câu. Trong quá trình training, vector của paragraph và vector từ đều được update.

![DM](doc2vec_dm.jpg)


Tóm lại: ta xem văn bản như là một từ, docID/paragraphID được biểu diễn dạng 1-hot, được embedded vào không gian vector.

# Setup

## Modules 

Mình sử dụng `gensim`, `gensim` được implement Word2vec lẫn Doc2vec, tài liệu rất dễ đọc.
Ngoài ra còn có `numpy` để thao tác trên array, `sklearn` cho các thuật toán phân lớp. 

In [9]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

# numpy
import numpy

# classifier
from sklearn.linear_model import LogisticRegression

# random
import random

## Data

Mình có các tập data để training, gồm training và testing như sau:

* `test-neg.txt`: 12500 negative movie reviews from the test data
* `test-pos.txt`: 12500 positive movie reviews from the test data
* `train-neg.txt`: 12500 negative movie reviews from the training data
* `train-pos.txt`: 12500 positive movie reviews from the training data
* `train-unsup.txt`: 50000 Unlabelled movie reviews

Trong mỗi file data, 1 đoạn văn bản trên một dòng:

```
once again mr costner has dragged out a movie for far longer than necessary aside from the terrific sea rescue sequences of...
this is an example of why the majority of action films are the same generic and boring there s really nothing worth watching...
this film grate at the teeth and i m still wondering what the heck bill paxton was doing in this film and why the heck does...
```

## Input data vào Doc2vec

Doc2vec của `gensim` input một object `LabeledSentence`, gồm 1 tập các từ kèm theo `label` (id của paragraph), có format như sau:

```python
[['word1', 'word2', 'word3', 'lastword'], ['label1']]
```

Ta sẽ viết class `LabeledLineSentence` để đọc data txt, yield ra object `LabeledSentence` để `gensim` có thể hiểu. 

In [4]:
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
        
        flipped = {}
        
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences
    
    def sentences_perm(self):
        shuffled = list(self.sentences)
        random.shuffle(shuffled)
        return shuffled

Ok bây giờ chúng ta feed data files vào `LabeledLineSentence`, `LabeledLineSentence` input 1 dict với key là tên file, value là `prefix` của các sentences trong văn bản.  

In [7]:
sources = {
    'data/test-neg.txt':'TEST_NEG',
    'data/test-pos.txt':'TEST_POS', 
    'data/train-neg.txt':'TRAIN_NEG', 
    'data/train-pos.txt':'TRAIN_POS', 
    'data/train-unsup.txt':'TRAIN_UNS'
}

sentences = LabeledLineSentence(sources)

# Model

## Building the Vocabulary Table

Trước tiên Doc2vec yêu cầu build Vocabulary table, 1 tập chứa tất cả các từ, lọc bỏ các từ trùng lặp, thực hiện một số thống kê. 
Chúng ta có một số tham số như sau:

* *min_count*: lọc bỏ tất cả các từ khỏi từ điển có số lần xuất hiện nhỏ hơn `min_count`.
* *window*: khoảng cách tối đa của từ hiện tại và từ predicted. Tương tự `window` trong Skip-gram model.
* *size*: số chiều của vector embedded. Thường từ trong khoảng 100-400 cho kết quả tối ưu. 
* *workers*: Số worker threads (set bằng số core của máy).

In [8]:
model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=7)

model.build_vocab(sentences.to_array())


IOError: [Errno 2] No such file or directory: 'train-neg.txt'

## Training Doc2Vec

Mình sẽ train model với 10 epochs. Nếu có thời gian bạn có thể chọn 20 hoặc 50 epochs. Mỗi epochs là một lần training trên toàn bộ dữ liệu.

Phần này có thể tốn rất nhiều thời gian, tùy cấu hình máy bạn như thế nào. 

In [10]:
for epoch in range(1):
    model.train(sentences.sentences_perm())

RuntimeError: you must first build vocabulary before training the model

## Inspecting the Model

Sau khi training, chúng ta có được các vector từ và vector văn bản. Tìm similarity của một từ (theo khoảng cách cosine).

In [ ]:
model.most_similar('good')

Vector của một đoạn văn trong tập negative reviews:

In [ ]:
model['TRAIN_NEG_0']

## Saving and Loading Models

Lưu xuống và tái sử dụng

In [ ]:
model.save('./imdb.d2v')

Khi sử dụng, model chỉ cần load lại 

In [ ]:
model = Doc2Vec.load('./imdb.d2v')

# Classifying Sentiments

Từ các vector văn bản trên, ta có thể sử dụng chúng để huấn luyện các bộ phân lớp. Trước tiên mình extract các vector này từ Doc2vec ra. Ở trên chúng ta có 25000 đoạn training reviews (12500 positive, 12500 negative), tức chúng ta sẽ có 25000 vector.

Chúng ta tạo 2 biến `train_arrays` và `train_labels` để lưu lại các vectors và labels tương ứng. 

In [ ]:
train_arrays = numpy.zeros((25000, 100))
train_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model[prefix_train_pos]
    train_arrays[12500 + i] = model[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0

`train_arrays` sẽ là một list các vector:

In [ ]:
print train_arrays

`train_labels` sẽ là một list các label tương ứng, label 1: positive, label 0: negative.

In [ ]:
print train_labels

## Testing Vectors

Tương tự với các vector để test

In [ ]:
test_arrays = numpy.zeros((25000, 100))
test_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i] = model[prefix_test_pos]
    test_arrays[12500 + i] = model[prefix_test_neg]
    test_labels[i] = 1
    test_labels[12500 + i] = 0

# Classification

Từ đây bạn có thể sử dụng bất cứ thuật toán phân lớp nào, bạn có thể thực hiện tiếp bước **model selection** đến khi nào đạt kết quả tốt nhất. Ở đây mình sử dụng **Logistic Regression** và **SVM**

In [ ]:
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

Độ chính xác của thuật toán:

In [ ]:
classifier.score(test_arrays, test_labels)

Ok, chúng ta có **87% accuracy** cho sentiment analysis với **Logistic Regression**.

Thử với **SVM**